In [158]:
import scipy.io
import numpy 
data = scipy.io.loadmat('Indian_pines_corrected.mat')  # imported data as dictionary
inp_table=data.pop('indian_pines_corrected') # extract specific key value pair
inp_table=inp_table.reshape(-1,200)  # flatning input data as 1D 
datao=scipy.io.loadmat('Indian_pines_gt.mat')  # imported ground truth as dictionary
gt=datao['indian_pines_gt']   # extract specific key value pair
gt=gt.reshape(-1,1)   # flatning ground truth data as 1D


In [159]:
indices=numpy.where(gt==0)[0]  # finding all indices with ground truth value 0


In [160]:
updated_gt=numpy.delete(gt,indices,0)  # Deleting entries with 0 as the output class from ground truth
updated_inp_table=numpy.delete(inp_table,indices,0) # Deleting entries with 0 as the output class from input data
updated_inp_table=updated_inp_table.astype(float) # Updating data type of input data as float
row,col=updated_inp_table.shape  # Storing rows and columns in input data

In [161]:
onehot_gt=numpy.zeros((row,16))  # Defining zeros numpy matrix (10249, 16)
for i in range(row):
    onehot_gt[i][updated_gt[i]-1]=1   # making 1 only at the class of the input


In [162]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(updated_inp_table,onehot_gt,test_size=0.5,random_state=0)  # Spliting train and test data


for i in range(x_train.shape[0]):
    x_train[i,:]=x_train[i,:]/numpy.max(x_train[i,:])  # Scaling input train data


for i in range(x_test.shape[0]):
    x_test[i,:]=x_test[i,:]/numpy.max(x_test[i,:])  # Scaling input test data


In [166]:


weights=numpy.zeros((200,16))    # Defining weight matrix (200,16) all zeros entry

from sklearn.preprocessing import normalize
for i in range(50000):
    y_cal=numpy.dot(x_train,weights)  # product of weights and input data
    
    expo_val=numpy.exp(y_cal)   # finding exponential of each value
                            
    train_row,train_col=y_train.shape  # storing rows and columns of data

    
    cost=(1/train_row)*(numpy.sum(-y_train*numpy.log(expo_val))) # finding cost 
   
    norm_expo_val = normalize(expo_val, axis=1, norm='l1')  # normalizing the exponantial values this will give probalility 
    
    if(i%1000==0):
        print("Cost: "+str(cost))   # printing cost after every 1000 iterations
        print("Squared ERROR  "+ str( (numpy.sum(numpy.square(y_train-norm_expo_val))/train_row*train_col)))  # printing squared error after  every 1000 iterations



   
    M=y_train-norm_expo_val  # defining M (10249, 16) matrix

    M=M.T
    grad=(numpy.dot(M,x_train))+(0.1*weights.T) #  finding gradient 
    grad=grad/train_row     

    weights=weights+(0.1*grad.T) # updating weights
    

Cost: 0.0
Squared ERROR  15.0
Cost: -2.5044098901119347
Squared ERROR  11.373305234383606
Cost: -2.9779960764009967
Squared ERROR  10.85579172215176
Cost: -3.3121371739003282
Squared ERROR  10.50200263527783
Cost: -3.575653803142197
Squared ERROR  10.227318185567578
Cost: -3.795578856589495
Squared ERROR  10.002192288034939
Cost: -3.985171367685795
Squared ERROR  9.814532136354025
Cost: -4.15200188926278
Squared ERROR  9.656230775618415
Cost: -4.30104446659424
Squared ERROR  9.5203504492246
Cost: -4.435847582295638
Squared ERROR  9.401470950763784
Cost: -4.559039102081398
Squared ERROR  9.295678635385615
Cost: -4.672606788965142
Squared ERROR  9.200207180856138
Cost: -4.77808031028288
Squared ERROR  9.113080060549539
Cost: -4.876655926552802
Squared ERROR  9.032851405537619
Cost: -4.969283586451934
Squared ERROR  8.958434879173394
Cost: -5.056728540140222
Squared ERROR  8.888992986147803
Cost: -5.139615485770478
Squared ERROR  8.8238649256227
Cost: -5.218460631562391
Squared ERROR  8.7

# Finding Test Error

In [168]:
y_cal_test=numpy.dot(x_test,weights)    
expo_val_test=numpy.exp(y_cal_test)
norm_expo_val_test = normalize(expo_val_test, axis=1, norm='l1')

cou=0    # Initilize count to zeoo                                           

p=numpy.argmax(norm_expo_val_test,axis=1)
q=numpy.argmax(y_test,axis=1)
for i in range(x_test.shape[0]):
    if(p[i]==q[i]):
        cou+=1       # increse count for each correct classification

print("Test Accuracy: "+str(100*cou/x_test.shape[0])) 
    

Test Accuracy: 64.62439024390244
